<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

%pip install inltk
import inltk 
from inltk.inltk import setup
from inltk.inltk import tokenize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Note: you may need to restart the kernel to use updated packages.


In [2]:
%cd /home/glitch/project/Sanskrit2mal

/home/glitch/project/Sanskrit2mal


In [3]:
%cd /home/glitch/project/Sanskrit2mal/parallel-corpus

mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

print(len(mal_lines))
print(len(sanskrit_lines))

/home/glitch/project/Sanskrit2mal/parallel-corpus
701
701


In [4]:
setup('ml')
setup('sa')

RuntimeError: This event loop is already running

Done!


In [5]:
def tokenizer_sa(sanskrit_lines):
    return [tokenize(sanskrit_lines,'sa')]

def tokenizer_ma(mal_lines):
    return [tokenize(mal_lines,'ml')]

In [46]:
import random
c = list(zip(sanskrit_lines, mal_lines))
random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:351]
train_text_mal = mal_lines[:351]

dev_text_sa = sanskrit_lines[:350]
dev_text_mal = mal_lines[:350]

In [47]:
print(len(train_text_mal))
print(len(dev_text_mal))

print(len(train_text_sa))
print(len(dev_text_sa))

351
350
351
350


In [48]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [49]:
# Reading the data and forming pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_mal[i]] for i in range(len(train_text_sa))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [50]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [51]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'mal', False)
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 351 sentence pairs
Trimmed to 351 sentence pairs
Counting words...
Counted words:
sanskrit 2378
mal 3489
['क्षिप्रं भवति धर्मात्मा शश्वच्छान्तिं निगच्छति। कौन्तेय प्रतिजानीहि न मे भक्तः प्रणश्यति।।', 'അവ\u200dന്\u200d വേഗത്തി\u200dല്\u200d ധര്\u200dമ്മാത്മാവായിത്തീ\u200dര്\u200dന്ന് ശാശ്വതമായ ശാന്തിയെ പ്രാപിക്കുന്നു. ഹേ അ\u200dര്\u200dജ്ജുനാ! എന്റെ ഭക്ത\u200dന്\u200d നശിക്കുന്നില്ലെന്ന് അറിഞ്ഞാലും.']


In [52]:
print(pairs[0])

['तानि सर्वाणि संयम्य युक्त आसीत मत्परः। वशे हि यस्येन्द्रियाणि तस्य प्रज्ञा प्रतिष्ठिता।।', 'അവയെല്ല\u200dാം സംയമനം ചെയ്തു യോഗയുക്തനായി എന്നില്\u200d ഭക്തിയോടുകൂടി ഇരിക്കുക. ആര്\u200dക്കു ഇന്ദ്രിയങ്ങള്\u200d വശത്താണോ അവന്റെ പ്രജഞ പ്രതിഷ്ഠിതമാണ്.']


# **Seq2Seq Model**
---
A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

# **Encoder**

In [53]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Decoder**

In [54]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Training**


In [55]:
import random
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(random.choice(list(lang.word2index.values())))    
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **Training the model**

In [56]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [57]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [58]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    #training_pairs = training_pairs1
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

# **Evaluation**

In [59]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [60]:
def evaluate_test(encoder, decoder):
  test = []
  %cd /home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
  with open('test_sanskrit.sa', 'r', encoding="utf8") as f:
    test_text = [x.replace('\n', '') for x in f.readlines()]
  for sa_text in test_text:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    # print('<', output_sentence)
    # print('')
    test.append(output_sentence)
  with open('translated_malayalam.txt', 'w', encoding="utf8") as f:
    for line in test:
        f.write(line + '\n')

In [61]:
def translator(encoder, decoder):
    test_text = 'धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः। मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय।।'

    output_words, attentions = evaluate(encoder, decoder, test_text)
    output_sentence = ' '.join(output_words)
    print(output_sentence)

In [62]:
 %cd /home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal


In [71]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 1024

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

5m 37s (- 78m 38s) (5000 6%) 3.2363
11m 31s (- 74m 54s) (10000 13%) 0.1733
17m 27s (- 69m 50s) (15000 20%) 0.0177
23m 22s (- 64m 16s) (20000 26%) 0.0049
29m 17s (- 58m 35s) (25000 33%) 0.0035
35m 13s (- 52m 49s) (30000 40%) 0.0028
41m 6s (- 46m 59s) (35000 46%) 0.0023
47m 0s (- 41m 8s) (40000 53%) 0.0019
52m 54s (- 35m 16s) (45000 60%) 0.0017
58m 48s (- 29m 24s) (50000 66%) 0.0015
64m 42s (- 23m 31s) (55000 73%) 0.0013
70m 35s (- 17m 38s) (60000 80%) 0.0012
76m 29s (- 11m 46s) (65000 86%) 0.0011
82m 24s (- 5m 53s) (70000 93%) 0.0010
88m 21s (- 0m 0s) (75000 100%) 0.0010


In [72]:
# saving the trained models
%cd /home/glitch/project/Sanskrit2mal
torch.save(encoder1.state_dict(), 'trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), 'trained_decoder.pth')

/home/glitch/project/Sanskrit2mal


In [73]:
evaluate_test(encoder1, attn_decoder1)

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal


In [74]:
%cd /home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
output_text=[]
with open ('translated_malayalam.txt','r',encoding="utf8") as f:
  output_text.extend([f.readlines()])
print(output_text)

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
[['ധൃതരാഷ്ട്രര്\u200d ചോദിച്ചു: സഞ്ജയ, ധ\u200dര്\u200dമ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്\u200d ഒരുമിച്ചു ചേര്\u200dന്നവരും യുദ്ധം ചെയ്യാന്\u200d കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്\u200dമാരും പാണ്ഡവന്\u200dമാരും എന്ത് ചെയ്തു? <EOS>\n', 'ഞാന്\u200d സര്\u200dവ്വരുടെയും ഹൃദയത്തില്\u200d സന്നിഹിതനാണ്. ഓര്\u200dമ്മ, അറിവ്, മറവി എന്നിവയുണ്ടാകുന്നതും എന്നില്\u200d നിന്നാണ്. എല്ലാ വേദങ്ങളിലൂടെ അറിയപ്പെടേണ്ടവന്\u200d ഞാനാണ്. വേദാന്തത്തിന്റെ കര്\u200dത്താവും വേദജ്ഞനും ഞാന്\u200d തന്നെയാണ്. <EOS>\n', 'ഹേ കൗന്തേയ, രജസ്സിന്റെ സ്വഭാവം രാഗമാണെന്നറിയൂ. അത് തൃഷ്ണ (ആഗ്രഹം) യെയും സംഗത്തെയും ജനിപ്പിക്കുകയും കര്\u200dമ്മത്തോടുള്ള സംഗത്താല്\u200d ദേഹിയെ ബന്ധിക്കുകയും ചെയ്യുന്നു. <EOS>\n', 'പാണ്ഡവ സൈന്യത്തില്\u200d ഭീമാര്\u200dജുനതുല്യരും ശൂരരും വലിയ വില്ലാളികളുമായ യുയുധാനനും വിരാടനും മഹാരഥനായ ദ്രുപദനും ദൃഷ്ടകേതുവും ചേകിതാനനും വീര്യവാനായ കാശിരാജാവും പുരുജിത്തും കുന്തിഭോജനും നരശ്രേഷ്ടനായ ശൈബ്യനും വിക്രമിയായ യുധാമന്യുവും വീര്യവാനായ ഉത്തമൌജസും സുഭദ്രാതനയന

In [75]:
translator(encoder1, attn_decoder1)

ധൃതരാഷ്ട്രര്‍ ചോദിച്ചു: സഞ്ജയ, ധ‍ര്‍മ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്‍ ഒരുമിച്ചു ചേര്‍ന്നവരും യുദ്ധം ചെയ്യാന്‍ കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്‍മാരും പാണ്ഡവന്‍മാരും എന്ത് ചെയ്തു? <EOS>


In [76]:
%cd /home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
import nltk
with open('test_malayalam.txt', 'r') as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

print("bleu", nltk.translate.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie))

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
bleu 7.298051926314169e-05
